In [310]:
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation

In [311]:
# creating empty data
n_cameras = 100
T = np.zeros((n_cameras, 3))
eR = np.zeros((n_cameras, 3))
print(T.shape)
print(eR.shape)
data = np.hstack((T, eR))
print(data.shape)
df = pd.DataFrame(data, columns=('x', 'y', 'z', 'pitch', 'yaw', 'roll')) #zxy coordinate order (colmap)

(100, 3)
(100, 3)
(100, 6)


In [312]:
#data manipulation
# translation (spiral)
a = 1
t = np.linspace(0, 6 * np.pi, n_cameras)
df.x = a*t*np.cos(t)
df.y = a*t*np.sin(t)
df.z = np.linspace(0, 6, n_cameras)
# angle (rotation based on the spiral)
df.yaw = np.rad2deg(t) + 90

In [313]:
df.loc[:, ['x', 'y', 'z']].head()

,x,y,z
0,0.000000,0.000000,0.000000
1,0.186959,0.036033,0.060606
2,0.353522,0.141529,0.121212
3,0.480523,0.308813,0.181818
4,0.551195,0.525563,0.242424


In [314]:
xyz = df.loc[:, ['x', 'y', 'z']].to_numpy(copy=True)
df.loc[:, ['z', 'x', 'y']] = xyz
df.yaw = -df.yaw # negate rotation
# angle conversions
euler_angles = np.array([df.pitch, df.yaw, df.roll]).transpose() # CHANGE pitch and yaw now are in correct order
r = Rotation.from_euler('xyz', euler_angles, degrees=True)
df['qx'], df['qy'], df['qz'], df['qw'] \
    = r.as_quat().transpose()

#filling out required data with none
df['image_id'] = np.arange(n_cameras)
df['camera_id'] = 1
df['image_name'] = 'image.png'


In [315]:
df.loc[:, ['x', 'y', 'z']].head()

,x,y,z
0,0.000000,0.000000,0.000000
1,0.036033,0.060606,0.186959
2,0.141529,0.121212,0.353522
3,0.308813,0.181818,0.480523
4,0.525563,0.242424,0.551195


In [316]:
# matrix conversion for the orientation to be ok
T = df.loc[:, ['x', 'y', 'z']].to_numpy()

T = r.apply(T)
T = -T # the final translation vector needs to be negated (colmap input requirenment)

df['tx'], df['ty'], df['tz'] = T.transpose()

In [317]:
#writing data
selected = df.copy()
selected = selected[['image_id', 'qw', 'qx', 'qy', 'qz', 'tx', 'ty', 'tz', 'camera_id', 'image_name']]
selected.to_csv('./raw_images.csv', sep=' ', header=False, index=False)

Rotation is ok, but the proper order shall be used (pitch, yaw, roll)

Translation vector shall be rotated by the rotation matrix (inverse of the inverse)

Final translation vector shall be negated (because of the output format)

Rotation angles shall be negated

Store xyz carla world coordinates shall be stored in zxy colmap world coordinates